In [12]:
from imutils.perspective import four_point_transform
from skimage.segmentation import clear_border
import numpy as np
import imutils
import cv2 as cv
from PIL import Image

In [36]:
def find_puzzle(image, debug=False):
	gray = cv.cvtColor(src=image, code=cv.COLOR_BGR2GRAY)
	blurred = cv.GaussianBlur(src=gray, ksize=(7,7), sigmaX=3)

	thresh = cv.adaptiveThreshold(src=blurred, maxValue=255, 
			       adaptiveMethod=cv.ADAPTIVE_THRESH_GAUSSIAN_C, 
				   thresholdType=cv.THRESH_BINARY, blockSize=11, C=2)
	thresh = cv.bitwise_not(src=thresh)  # Contours become white
		
	contours = cv.findContours(image=thresh.copy(), mode=cv.RETR_EXTERNAL, method=cv.CHAIN_APPROX_SIMPLE)
	contours = imutils.grab_contours(cnts=contours)
	contours = sorted(contours, key=cv.contourArea, reverse=True)

	puzzle_contour = None
	for contour in contours:
		perimeter = cv.arcLength(curve=contour, closed=True)
		estimate = cv.approxPolyDP(curve=contour, epsilon=0.02 * perimeter, closed=True)

		if len(estimate) == 4:
			puzzle_contour = estimate
			break

	if puzzle_contour is None:
		raise Exception(("Could not find Sudoku puzzle outline. "
			"Try debugging your thresholding and contour steps."))

	puzzle = four_point_transform(image=image, pts=puzzle_contour.reshape(4, 2))
	warped = four_point_transform(image=gray,  pts=puzzle_contour.reshape(4, 2))
	
	if debug:
		cv.imshow("Puzzle Input", image)
		cv.waitKey(0)

		cv.imshow("Puzzle Gray", gray)
		cv.waitKey(0)

		cv.imshow("Puzzle Blurred", blurred)
		cv.waitKey(0)

		cv.imshow("Puzzle Thresh", thresh)
		cv.waitKey(0)

		output = image.copy()
		cv.drawContours(output, [puzzle_contour], -1, (0, 255, 0), 2)
		cv.imshow("Puzzle Outline", output)
		cv.waitKey(0)

		cv.imshow("Puzzle Transform", puzzle)
		cv.waitKey(0)

		cv.imshow("Puzzle Warped", warped)
		cv.waitKey(0)
	return (puzzle, warped)

In [21]:
image = cv.imread('../input/picca.webp')

In [37]:
find_puzzle(image, debug=True)

(array([[[118, 120, 147],
         [116, 118, 146],
         [122, 123, 152],
         ...,
         [ 99, 100, 123],
         [119, 120, 142],
         [155, 157, 177]],
 
        [[ 90,  91, 121],
         [ 85,  86, 117],
         [ 83,  84, 116],
         ...,
         [ 97,  98, 121],
         [118, 121, 142],
         [155, 158, 178]],
 
        [[ 68,  68, 100],
         [ 62,  63,  95],
         [ 63,  63,  96],
         ...,
         [ 94,  95, 118],
         [117, 120, 141],
         [154, 157, 177]],
 
        ...,
 
        [[124, 120, 145],
         [ 73,  68,  95],
         [ 64,  57,  86],
         ...,
         [ 94,  88, 115],
         [ 98,  92, 119],
         [105,  99, 126]],
 
        [[129, 123, 148],
         [ 74,  68,  94],
         [ 57,  51,  78],
         ...,
         [118, 112, 136],
         [122, 116, 140],
         [127, 122, 144]],
 
        [[152, 146, 170],
         [105, 100, 123],
         [ 94,  88, 112],
         ...,
         [191, 186, 207],
  